In [11]:
import os, glob, sys, numpy as np
import cv2
import shutil

from PIL import Image
from tqdm import tqdm
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


# images.shape = (None, 224, 244). value = 0~225
def equalize_images(images):
    return np.array([cv2.equalizeHist(image) for image in images])
    
def generate_images_for_data_augmentation(original_path, output_path, prefix, max_gen_count):
    #create_output_path(output_path)
    
    file_list = os.listdir(original_path)
    
    datagen = ImageDataGenerator(  
                rotation_range=10,
                width_shift_range=0.01,
                height_shift_range=0.01,
                #shear_range=0.2,
                zoom_range=0.1,
                horizontal_flip=True,
                fill_mode="nearest")
    
    for filename in tqdm(file_list) :
        # copy
        shutil.copyfile(original_path + filename, output_path + filename)
        
        # generates
        img = load_img(original_path  + filename)  # this is a PIL image
        
        img_data = img_to_array(img) # this is a Numpy array
        img_data = img_data.reshape((1,) + img_data.shape)  # this is a Numpy array 

        # the .flow() command below generates batches of randomly transformed images
        # and saves the results to the `preview/` directory
        generated_count = 0
        
        for batch in datagen.flow(img_data, batch_size = 1,
                                  save_to_dir=output_path, save_prefix=prefix, save_format="jpg"):
            generated_count += 1

            if generated_count > max_gen_count:
                break  # otherwise the generator would loop indefinitely

In [12]:
import shutil

# 이미지 path
fracture_path = './x-ray/fracture_resize_reverse_crop/'
normal_path = './x-ray/Normal_resize_reverse_crop/'

# 이미지 증강 path
generate_fracture_path = './x-ray/generate_fracture/'
generate_normal_path ='./x-ray/generate_normal/'
path_category = [generate_fracture_path, generate_normal_path]

# path folders Make 
for idx, folder in enumerate(path_category) :
    if os.path.isdir(folder) : 
        shutil.rmtree(folder)
        os.mkdir(folder)
        print('Folders Make!!')
    else : 
        os.mkdir(folder)
        print('Folders Make!!')


Folders Make!!
Folders Make!!


In [13]:
# 이미지 증강
generate_images_for_data_augmentation(fracture_path, generate_fracture_path, '', 10)
generate_images_for_data_augmentation(normal_path, generate_normal_path, '', 10)

100%|██████████| 695/695 [00:27<00:00, 25.43it/s]


In [14]:
# GAN_Data_set - facture_only
X = []

for file_name in tqdm(glob.glob(generate_fracture_path + '*.jpg')) :
    img = Image.open(file_name).convert('L')
    data = np.array(img)
    # equalization
    data = equalize_images(data) 
    X.append(data)

fracture_train = np.array(X)
# normalization -1 ~ 1
fracture_train = (fracture_train - 127.5) / 127.5
print(fracture_train.shape)

100%|██████████| 8032/8032 [00:05<00:00, 1604.07it/s]
(8032, 128, 128, 1)


In [15]:
np.save('./npy/fracture_train.npy',fracture_train)

In [16]:
# normal data set 
X_ = []

for file_name in tqdm(glob.glob(generate_normal_path + '*.jpg')) :
    img = Image.open(file_name).convert('L')
    data = np.array(img)
    # equalization
    data = equalize_images(data)
    X_.append(data)

normal_train = np.array(X_)
# normalization -1 ~ 1
normal_train = (normal_train - 127.5) / 127.5
print(normal_train.shape)

100%|██████████| 6032/6032 [00:05<00:00, 1139.29it/s]
(6032, 128, 128, 1)


In [17]:
np.save('./npy/normal_train.npy',normal_train)

In [18]:
# Classfication 사용할 Data set 만들기
# fracture 와 normal data 1:1 비율

# normal_train 개수 확인 
train_len = normal_train.shape[0]
normal_train_x = normal_train

# facture data 개수를 Normal_data 개수와 맞추기 
fracture_train_x = fracture_train[ : train_len]

# y_label create
normal_train_y = np.zeros((train_len, 1)) # 비골절 0
fracture_train_y = np.ones((train_len, 1)) # 골절 1

print('normal data',normal_train_x.shape, normal_train_y.shape)
print('facture data',fracture_train_x.shape, fracture_train_y.shape)

normal data (6032, 128, 128, 1) (6032, 1)
facture data (6032, 128, 128, 1) (6032, 1)


In [19]:
# fracture + normal 
x = np.append(normal_train_x, fracture_train_x, axis = 0)
y = np.append(normal_train_y, fracture_train_y, axis = 0)

# data shuffle
shuffled_index = np.random.permutation(x.shape[0])
x = x[shuffled_index,:,:]
y = y[shuffled_index]

# data ready 
split_index = int(x.shape[0] * 0.9)
X_train_c, X_test = x[:split_index], x[split_index:]
Y_train_c, Y_test = y[:split_index], y[split_index:]

print('X_train', X_train_c.shape)
print('X_test ', X_test.shape)
print('Y_train', Y_train_c.shape)
print('Y_test ', Y_test.shape)

print(Y_test[:5]) # 0과 1이 섞여 있으면 suffle 성공

X_train (10857, 128, 128, 1)
X_test  (1207, 128, 128, 1)
Y_train (10857, 1)
Y_test  (1207, 1)
[[0.]
 [1.]
 [1.]
 [0.]
 [0.]]


In [20]:
xy = (X_train_c, X_test, Y_train_c, Y_test)
np.save('./npy/classify_train.npy', xy)